# Natural Language Processing con spaCy
**spaCy** è una delle librerie più avanzate e popolari per il Natural Language Processing (NLP) in Python. Creata per essere veloce, scalabile e facile da usare, spaCy offre strumenti di alto livello per l'analisi e la comprensione del linguaggio naturale.

Installiamo spaCy.

In [ ]:
!pip install spacy==3.8.2

In spaCy, un modello contiene tutte le informazioni necessarie per eseguire task specifici di NLP. Ogni modello è una pipeline addestrata in grado di elaborare il testo in una determinata lingua e fornire risultati strutturati.

spaCy offre pipeline per molte lingue diverse e con diversa grandezza, velocità, e accuratezza.

Per poter utilizzare una pipeline abbiamo bisogno di scaricare il relativo modello, proviamo a scaricare il modello `en_core_web_lg`.

https://spacy.io/models

In [ ]:
!python -m spacy download en_core_web_lg

Per utilizzare il modello scaricato dobbiamo caricarlo usando `spacy.load`. Questo ci restituirà una pipeline, che solitamente chiamiamo `nlp`, contenente tutte le componenti e le informazioni necessarie a processare il testo.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

Vediamo quali componenti contiene la nostra pipeline.
https://spacy.io/usage/processing-pipelines

In [ ]:
nlp.pipe_names

## Tokenizzazione
La tokenizzazione è il primo passo nella maggior parte dei processi NLP. Consiste nel suddividere il testo in unità più piccole, chiamate token. Questi possono essere parole, frasi o persino caratteri. È una tecnica fondamentale perché la maggior parte delle analisi linguistiche richiede di lavorare con unità discrete del testo, piuttosto che con interi blocchi.

Tokenizzare un testo potrebbe sembrare un task banale, ma in realtà vengono applicate regole specifiche per ogni lingua. Ad esempio, la punteggiatura va normalmente separata dalle parole - ma nel caso di "U.K." questo deve rimanere un token singolo.

Passando una stringa di testo alla pipeline `nlp` ci viene restituito un documento processato, di classe `Doc` che chiamiamo `doc`. `doc` è una sequenza di token, ossia di oggetti di classe `Token`, che hanno diversi attributi.

Per vedere i token che compongono il nostro testo originale ci basta iterare su `doc` e stampare i token.

In [ ]:
text = "Hello World!"
doc = nlp(text)

for token in doc:
    print(token.text)

Nonostante `doc` sia processato, contiene ancora tutte le informazioni del testo originale.

Possiamo, ad esempio, ricostruire il testo originale accedendo all'attributo `.text_with_ws` che conserva il testo del token insieme agli spazi adiacenti originali.

In [ ]:
"".join([token.text_with_ws for token in doc])

Possiamo anche recuperare la posizione originale di ogni token all'interno del testo usando l'attributo `idx` contenente l'indice del token.

In [ ]:
for token in doc:
    print(f"Token: {token.text}, Index: {token.idx}")

In generale, ogni token possiede diversi attributi:


* `.text`: il token così come compare nel testo
* `.idx`: l'indice del token all'interno del testo
* `.lemma_`: il token nella sua forma canonica, quella che troveremmo all'interno del dizionario
* `.is_punct`: il token è un segno di punteggiatura?
* `.is_space`: il token è uno spazio?
* `.shape_`: trasforma il token per mostrare le sue caratteristiche ortografiche
* `pos_`: Universal POS tag (https://universaldependencies.org/u/pos/) del token, specifica il ruolo del token dal punto di vista grammaticale
* `tag_`: Penn Treebank POS tag (https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), più specifico degli Universal POS tag. Ad esempio, questo tipo di POS tag è in grado di distinguere il tempo verbale e i nomi al singolare e al plurale


In [ ]:
text = "On January 12th, I will be in the U.K."
doc = nlp(text)

for token in doc:
    print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}".format(
        token.text,
        token.idx,
        token.lemma_,
        token.is_punct,
        token.is_space,
        token.shape_,
        token.pos_,
        token.tag_
    ))

## Sentence Segmentation
Il processo di sentence segmentation consiste nell'identificare i confini tra le frasi in un testo continuo, in modo da poterle dividere e analizzare singolarmente.

La classe `Doc` ci consente di accedere alle singole frasi che compongono il testo.

Per farlo, ci basta iterare sull'attributo `.sents`.

In [ ]:
text = "This is a sentence. This is another sentence."
doc = nlp(text)

for i, sent in enumerate(doc.sents):
    print(f"Sentence {i}: {sent}")

## Rimozione di caratteri speciali e stopword
La rimozione dei caratteri speciali e delle stopword (parole molto frequenti ma poco informative, come "il", "e", "di") è un passaggio di pulizia del testo. Serve a ridurre il rumore nei dati, concentrandosi sulle parole che portano più informazioni utili per l'analisi.

Partiamo dalla rimozione dei caratteri speciali.

Possiamo filtrare i caratteri speciali utilizzando le proprietà di ogni token. I caratteri speciali come punteggiatura, numeri o caratteri non alfanumerici possono essere infatti filtrati sfruttando i seguenti attributi dei token:
* `.is_punct`: per identificare la punteggiatura.
* `.is_alpha`: per identificare le parole composte da caratteri alfabetici.
* `.is_digit`: per identificare i numeri.
* `.is_space`: per identificare gli spazi.

In [ ]:
text = "Sample text with numbers (123), symbols @$%^&* and punctuation :;,."
doc = nlp(text)

clean_text = [token.text for token in doc if token.is_alpha]
print("Tokens without numbers, symbols, and punctuations:", clean_text)

In questo modo abbiamo ottenuto una lista di token. Se vogliamo ricostruire il testo mantenendo solo i token alfabetici possiamo usare `token.text_with_ws` come abbiamo fatto in precedenza e filtrare come abbiamo appena visto.

In [ ]:
"".join([token.text_with_ws for token in doc if token.is_alpha])

La rimozione delle stopword consiste nel rimuovere tutte quelle parole molto comuni in una determinata lingua. Si basa sull'idea che queste parole molto comuni non siano utili all'analisi del testo e si possono quindi rimuovere per ridurre il rumore.

In [ ]:
text = "The movie I saw last night was not that good"
doc = nlp(text)

clean_text = [token.text for token in doc if not token.is_stop]
print("Tokens without stopwords:", clean_text)


È importante notare che non esiste una vera e propria definizione di cosa sia una stopword, per questo motivo non esiste una singola lista universale di stopword per una determinata lingua. Le stopword più opportune da utilizzare variano di caso in caso, per questo motivo è possibile personalizzarle.

Vediamo come aggiungere e rimuovere stopword.

In [ ]:
# Aggiunta di una stopword
nlp.vocab["saw"].is_stop = True

# Rimozione di una stopword
nlp.vocab["not"].is_stop = False

Vediamo come cambia il risultato con le nuove stopword.

In [ ]:
text = "The movie I saw last night was not that good"
doc = nlp(text)

clean_text = [token.text for token in doc if not token.is_stop]
print("Tokens without stopwords:", clean_text)

Come prima, possiamo ricostruire il testo originale senza le stopword.

In [ ]:
"".join([token.text_with_ws for token in doc if not token.is_stop])

## Lemmatizzazione e Stemming
La lemmatizzazione e lo stemming sono tecniche per ridurre le parole alle loro forme base.

Lo stemming taglia il suffisso delle parole per ottenere una radice comune (es. "correndo" -> "corr").
La lemmatizzazione restituisce la forma grammaticale corretta, basandosi sul contesto (es. "correndo" -> "correre").
Sono utili per analisi testuali più accurate e per uniformare le varianti di una stessa parola.

Partiamo dalla lemmatizzazione. Come abbiamo visto in precedenza, possiamo accedere al lemma corrispondente ad ogni token grazie all'attributo `.lemma_`.

In [ ]:
text = "The boys ran quickly through the park while watching the dogs play."
doc = nlp(text)

lemmas = [token.lemma_ for token in doc]
print("Lemmas:", lemmas)

Per quanto riguarda lo stemming, questo non è nativamente supportato da spaCy. Per questo motivo ricorriamo ad uno stemmer presente in un'altra libreria per il Natural Language Processing, chiamata **Natural Language Toolkit** (**NLTK**).

Installiamo NLTK.

In [ ]:
!pip install nltk

Proviamo il Porter Stemmer.

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

stems = [stemmer.stem(token.text) for token in doc]
print("Stems:", stems)

## Part-of-Speech (POS) Tagging
Il POS tagging assegna a ogni parola il suo ruolo grammaticale, come sostantivo, verbo o aggettivo. Questa analisi permette di capire meglio il significato delle parole e le relazioni tra di esse, rendendola una base fondamentale per molte tecniche avanzate, come la comprensione del contesto.

La classe `Doc` ci consente di accedere ai POS tag del testo. Come abbiamo visto in precedenza, possiamo usare gli attributi `pos_` e `tag_` dei singoli token per accedere agli Universal POS tag e ai Penn Treebank POS tag.

In [ ]:
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

print([(token.text, token.pos_) for token in doc])
print([(token.text, token.tag_) for token in doc])

## Named Entity Recognition (NER)
La NER identifica e classifica entità nominate in un testo, come persone, luoghi, aziende, date o valori numerici. Questa tecnica è cruciale per estrarre informazioni strutturate da testi non strutturati, con applicazioni in campi come il monitoraggio delle notizie o l'analisi di recensioni.

Le entità che possiamo riconoscere in spaCy sono:
* PERSON:      People, including fictional.
* NORP:        Nationalities or religious or political groups.
* FAC:         Buildings, airports, highways, bridges, etc.
* ORG:         Companies, agencies, institutions, etc.
* GPE:         Countries, cities, states.
* LOC:         Non-GPE locations, mountain ranges, bodies of water.
* PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
* EVENT:       Named hurricanes, battles, wars, sports events, etc.
* WORK_OF_ART: Titles of books, songs, etc.
* LAW:         Named documents made into laws.
* LANGUAGE:    Any named language.
* DATE:        Absolute or relative dates or periods.
* TIME:        Times smaller than a day.
* PERCENT:     Percentage, including ”%“.
* MONEY:       Monetary values, including unit.
* QUANTITY:    Measurements, as of weight or distance.
* ORDINAL:     “first”, “second”, etc.
* CARDINAL:    Numerals that do not fall under another type.

La classe `Doc` ci consente di accedere alle entità riconosciute all'interno del nostro testo. Le entità rilevate nel testo sono contenute nell'attributo `.ents`.

Per accedere all'etichetta di un'entità si può utilizzare l'attributo `.label_`.
Per accedere all'indice di inizio e di fine di ogni entità si possono usare gli attributi `.start_char` ed `.end_char`.

In [ ]:
text = "On January 12th, I will be in Rome."
doc = nlp(text)

for ent in doc.ents:
    print(f"Entity text: {ent.text} \t Label: {ent.label_} \t Start index: {ent.start_char}  \t End index: {ent.end_char}")

I tag IOB (Inside, Outside, Beginning) sono un sistema utilizzato per annotare le entità nominate ed indicano la posizione di ciascun token all'interno di un'entità nominata.

Possiamo accedere ai tag IOB dei token grazie all'attributo `.ent_iob_`.

In [ ]:
text = "Next week I'll be in Madrid."
doc = nlp(text)

iob_tagged = [
    (
        token.text,
        token.tag_,
        "{0}-{1}".format(token.ent_iob_, token.ent_type_) if token.ent_iob_ != 'O' else token.ent_iob_
    ) for token in doc
]
print(iob_tagged)

Ogni modello disponibile in spaCy è in grado di riconoscere determinati tipi di entità. È possibile vedere la lista completa di entità supportate da ogni modello qui https://spacy.io/models/en.

Grazie a displaCy possiamo visualizzare grafici e diagrammi che rappresentano visivamente le informazioni linguistiche. È uno strumento interattivo che può essere utilizzato per visualizzare i risultati del processing NLP in modo intuitivo, aiutando nella comprensione delle relazioni tra le parole in una frase e dei concetti estratti dal testo.

Usiamo displaCy per visualizzare le entità riconosciute.
* `displacy.render` è il metodo che genera la visualizzazione.
* `doc` è l'oggetto di classe `Doc` che contiene il testo elaborato.
* `style='ent'` indica che vogliamo visualizzare le entità nominate nel testo.
* `jupyter=True` permette la visualizzazione all'interno di un Jupyter Notebook.

In [ ]:
from spacy import displacy

text = "I just bought 2 shares at 9 a.m. because the stock went up 30% in just 2 days, according to the WSJ"
doc = nlp(text)

displacy.render(doc, style='ent', jupyter=True)

## Estrazione Noun Chunks
Un noun chunk è un gruppo di parole che ruotano attorno a un sostantivo, spesso includendo l'aggettivo, e altre parole che modificano il sostantivo.

La classe `Doc` ci consente di accedere ai noun chunks individuati nel nostro testo.

Possiamo accedere ai noun chunks riconosciuti nel testo tramite l'attributo `.noun_chunks`. Per ogni noun chunk stampiamo il testo grazie all'attributo `.text`, e la "root" - cioè il sostantivo principale da cui dipendono gli altri componenti del noun chunk - grazie all'attributo .`root_text`.

In [ ]:
text = "Wall Street Journal just published an interesting piece on large language models"
doc = nlp(text)

for chunk in doc.noun_chunks:
    print(chunk.text)
    print(f"Root: {chunk.root.text} \n")

## Dependency Parsing
Il dependency parsing analizza la struttura sintattica di una frase, identificando le relazioni grammaticali tra le parole. Ad esempio, collega un verbo al suo soggetto o oggetto. Questa analisi è fondamentale per comprendere il significato profondo di un testo, soprattutto per applicazioni che richiedono una comprensione avanzata. Ogni parola della frase è associata a un'altra parola chiamata "head", e l'intera frase è organizzata come una struttura di dipendenze.

Possiamo accedere alle informazioni relative al dependency parsing accedendo ai seguenti attributi dei token nella frase:
* `token.head.text`: Il testo della head del token. La head è il token principale o centrale da cui il token in esame dipende grammaticalmente.
* `token.dep_`: Il ruolo di dipendenza del token nella frase. Indica come il token è grammaticalmente collegato alla sua head.
* `token.head.tag_`: Il POS tag della head del token, ovvero il tipo grammaticale della parola centrale da cui il token dipende.

In [ ]:
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

for token in doc:
    print("{0}/{1} <--{2}-- {3}/{4}".format(
        token.text, token.tag_, token.dep_, token.head.text, token.head.tag_))

Questa visualizzazione non è molto leggibile, possiamo ottenerne una migliore usando ancora displaCy. In questo caso useremo `style='dep'` per indicare che vogliamo stampare l'albero delle dipendenze.

In [ ]:
from spacy import displacy

text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

displacy.render(doc, style='dep', jupyter=True)

### Analisi del dependency parsing della frase: "The quick brown fox jumps over the lazy dog"


1. **"The" (det)**
   - **Descrizione**: "The" è un determinante che precede il sostantivo "fox" per specificarlo. In questa frase, "The" è un `det` (determinante) e dipende da "fox", che è il sostantivo principale del gruppo nominale "The quick brown fox".

2. **"quick" (amod)**
   - **Descrizione**: "quick" è un aggettivo che modifica il sostantivo "fox". Indica una qualità della "fox", quindi la relazione di dipendenza tra "quick" e "fox" è `amod` (modificatore aggettivale). In altre parole, "quick" aggiunge una descrizione al sostantivo "fox".

3. **"brown" (amod)**
   - **Descrizione**: "brown" è un altro aggettivo che modifica "fox". Come "quick", "brown" è anche un modificatore aggettivale che descrive il sostantivo "fox". Insieme a "quick", forma una descrizione più completa della "fox".

4. **"fox" (nsubj)**
   - **Descrizione**: "fox" è il soggetto della frase, cioè colui che esegue l'azione del verbo. La relazione di dipendenza di "fox" è `nsubj` rispetto al verbo "jumps". In altre parole, "fox" è il soggetto che compie l'azione di "jumps".

5. **"jumps" (ROOT)**
   - **Descrizione**: "jumps" è il verbo principale della frase ed è la radice (`root`) della struttura. La radice è il verbo che esprime l'azione centrale della frase. Tutte le altre parole dipendono direttamente o indirettamente da "jumps".

6. **"over" (prep)**
   - **Descrizione**: "over" è una preposizione che introduce un complemento preposizionale. In questa frase, "over" stabilisce una relazione spaziale, dicendo dove la "fox" compie l'azione del verbo "jumps". La preposizione "over" dipende dal verbo "jumps", formando una struttura di dipendenza preposizionale.

7. **"the" (det)**
   - **Descrizione**: "the" è un determinante che precede il sostantivo "dog", specificandolo. In questa frase, "the" è un `det` e dipende dal sostantivo "dog", che è il complemento dell'azione "jumps".

8. **"lazy" (amod)**
   - **Descrizione**: "lazy" è un aggettivo che modifica il sostantivo "dog". Descrive la qualità del cane, quindi la relazione di dipendenza tra "lazy" e "dog" è `amod` (modificatore aggettivale). Insieme a "the", forma una descrizione completa di "dog".

9. **"dog" (pobj)**
   - **Descrizione**: "dog" è l'oggetto del complemento preposizionale "over". La preposizione "over" stabilisce una relazione tra "fox" (il soggetto) e "dog" (l'oggetto) tramite il movimento sopra di "dog". La relazione di dipendenza tra "dog" e "over" è `pobj` (oggetto preposizionale).

## Sentiment Analysis
La sentiment analysis è una tecnica di NLP utilizzata per classificare le emozioni espresse in un testo. Questo processo permette di determinare se il sentiment di un messaggio è positivo, negativo o neutro e di quantificare l'intensità dell'emozione.

Dato che spaCy non dispone di strumenti di sentiment analysis, utilizziamo un modello specifico per la sentiment analysis, chiamato **VADER**.

Installiamo VADER.

In [ ]:
!pip install vaderSentiment==3.3.2

Vediamo ora come utilizzare VADER per ottenere il sentiment di una frase.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentiment_analyzer = SentimentIntensityAnalyzer()

text = "The product quality is excellent, it is a really good product. Unfortunately, I'm disappointed because the delivery was very slow."
sentiment = sentiment_analyzer.polarity_scores(text)

print("Sentiment Analysis:", sentiment)

Possiamo combinare VADER con la sentence segmentation di spaCy per ottenere il sentiment delle singole frasi nel testo.

In [ ]:
doc = nlp(text)

for sent in doc.sents:
    sent_sentiment = sentiment_analyzer.polarity_scores(sent.text)
    print(f"Sentiment for '{sent.text}': {sent_sentiment}")

Oppure possiamo estendere la classe `Span` fornendole la capacità di fare Sentiment Analysis delle frasi in un testo.

Per farlo, dobbiamo aggiungere un nuovo attributo alla classe `Span` che conterrà il sentiment della frase.

In [ ]:
print(type(list(doc.sents)[0]))

In [ ]:
def compute_sentiment_scores(sent):
    return sentiment_analyzer.polarity_scores(sent.text)

from spacy.tokens import Span
Span.set_extension('sentiment_score', getter=compute_sentiment_scores, force=True)

In [ ]:
doc = nlp(text)
for sent in doc.sents:
  print(f"Sentiment for '{sent.text}': {sent._.sentiment_score}")

## Componenti Custom: Esempio con WordNet
Possiamo creare delle nostre componenti custom per le pipeline, dotate delle funzionalità di cui abbiamo bisogno. È possibile aggiungere queste componenti ad una pipeline esistente.

Creiamo una componente che ci consenta di estrarre la definizione di una parola usando **WordNet**. WordNet è un dizionario elettronico per la lingua inglese, organizzato in modo da catturare le relazioni semantiche e lessicali tra le parole.

Importiamo wordnet da NLTK.

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from spacy.tokens import Token
from spacy.language import Language

Scarichiamo il dizionario.

In [ ]:
nltk.download('wordnet')

Per ottenere la definizione di una parola da WordNet abbiamo bisogno di identificare il suo *synset*. Un synset è un insieme di sinonimi, ogni synset ha quindi un proprio significato ed una propria definizione.

Per assegnare una parola ad un synset abbiamo bisogno di capire in che modo questa viene utilizzata all'interno di una frase, cosa che possiamo fare usando i POS tag. WordNet utilizza un proprio sistema di POS tag, equivalente a quelli Penn Treebank. Dato che la nostra pipeline possiede già la capacità di estrarre questi POS tag, ci basta scrivere una funzione per convertirli.

In [ ]:
# Function to map Penn Treebank POS tags to WordNet POS tags
def penn_to_wordnet(tag):
    if tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('R'):
        return 'r'  # Adverb
    return None  # No matching WordNet POS

Adesso possiamo usare i POS tag di WordNet per ricavare il synset di un token ed estrarre la sua definizione.

In [ ]:
text = "I love dogs"
doc = nlp(text)

for token in doc:
    # Convert spaCy's POS tag to WordNet POS tag
    wn_pos = penn_to_wordnet(token.tag_)
    if wn_pos is None:
        continue  # Skip tokens with incompatible POS tags
    # Get the first WordNet synset (if available)
    synsets = wn.synsets(token.lemma_, wn_pos)
    synset = synsets[0]  # Use the first synset
    definition = synset.definition(lang="eng")
    print(f"Token: {token.lemma_}\nDefinition: {definition}\n")

Ora, proviamo a creare una componente custom per dare ad una pipeline la capacità di estrarre la definizione di un token.

Per farlo creiamo la classe `WordnetDefinition`, la classe rappresenta una componente custom che:
* Aggiunge un nuovo attributo custom all'oggetto `Token`, chiamato `definition`.
* Processa ogni token nel documento preprocessato per ottenere e conservare la sua definizione.

In [ ]:
# Custom spaCy pipeline component to add WordNet synsets
class WordnetDefinition:
    def __init__(self, nlp):
        # Add a custom attribute 'definition' to spaCy's Token object
        Token.set_extension('definition', default=None, force=True)

    def __call__(self, doc):
        for token in doc:
            # Convert spaCy's POS tag to WordNet POS tag
            wn_pos = penn_to_wordnet(token.tag_)
            if wn_pos is None:
                continue  # Skip tokens with incompatible POS tags
            # Get the first WordNet synset (if available)
            synsets = wn.synsets(token.text, wn_pos)
            if synsets:
                synset = synsets[0]  # Use the first synset
                token._.definition = synset.definition(lang="eng")  # Get the definition in English
        return doc

Per poter aggiungere la nostra nuova componente ad una pipeline di spaCy facendo riferimento al suo nome dobbiamo "registrare" la nuova componente. Questo è necessario per dare a spaCy la capacità di creare la nuova componente quando chiediamo di aggiungerla ad una pipeline.

In [ ]:
# Register the pipeline component in spaCy
# This is the function that spaCy calls when you add the "wordnet_definition" component to a pipeline
@Language.factory("wordnet_definition")
def create_wordnet_definition_component(nlp, name):
    return WordnetDefinition(nlp)

Aggiungiamo la componente appena creata ad una pipeline e testiamola.

In [ ]:
print(nlp.pipeline)

In [ ]:
nlp.add_pipe("wordnet_definition", last=True)
print(nlp.pipeline)

In [ ]:
text = "Paris is the awesome capital of France."
doc = nlp(text)

for token in doc:
    print(f"Token: {token.lemma_}\nDefinition: {token._.definition}\n")